<a href="https://colab.research.google.com/github/ankit-rathi/Tradevesting/blob/main/myProspects_MultiTimeFrame_Signals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import pytz
tz_NY = pytz.timezone('Asia/Kolkata')
datetime_NY = datetime.datetime.now(tz_NY)
print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

gen_output = 0

Run date time (IST): 2024-05-11 15:39:03


In [2]:
!pip install mplfinance

In [3]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
#from google.colab import drive
#drive.mount('/content/drive')

#mypf = pd.read_csv('/content/drive/My Drive/data/stocks/myProspectsScrips.csv')

back_date = '31-Dec-2025'

df_mypfs = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting/main/data/myPortfolioStocks.csv')
df_mypps = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting/main/data/myProspectsScrips.csv')

df_mypf = pd.merge(df_mypfs, df_mypps, on="Symbol")

#myts = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Tradevesting/myTransactionsReport.csv')

ar_pf = df_mypf[df_mypf['InPortfolio'] == 'DM']

sv_pf = df_mypf[df_mypf['InPortfolio'] == 'SV']


def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'ADANITRANS', 'SREERAYALK', 'SCHAEFFLER ', 'EQUITAS']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

all_prospects = get_stock_ids(df_mypps)
ar_stocks = get_stock_ids(ar_pf)
sv_stocks = get_stock_ids(sv_pf)
#df_stocks = ar_pf.append(sv_pf)
df_stocks = pd.concat([ar_pf,sv_pf], ignore_index = True)
df_mypps['InPortfolio'] = None
df_mypps.loc[(df_mypps['Symbol'].isin(sv_stocks)), 'InPortfolio'] = 'SV'
df_mypps.loc[(df_mypps['Symbol'].isin(ar_stocks)), 'InPortfolio'] = 'DM'

df_mypps = df_mypps[df_mypps['TFrame'] == 'W']

In [4]:
import pandas as pd
import mplfinance as mpf
import yfinance as yfin
#import talib
yfin.pdr_override()

def get_mtm_data(symbol, tf):
  if tf == 'D':
    intvl, sma, lma, xlma, prd, start_date, tfc = '1d', 20, 50, 200, '2y', '2023-01-01','Daily'
  elif tf == 'W':
    intvl, sma, lma, xlma, prd, start_date, tfc = '1wk', 5, 10, 30, '6y', '2019-01-01','Weekly'
  else:
    intvl, sma, lma, xlma, prd, start_date, tfc = '1mo', 2, 5, 10, '10y', '2013-01-01','Monthly'

  # Sample data with OHLCV (Open, High, Low, Close, Volume) prices
  # Sample data
  df = yfin.Ticker(symbol+'.NS').history(period=prd, interval=intvl)[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
  df = df[df.index < back_date]
  #df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
  #df = pd.DataFrame(data)
  #df.set_index('Date', inplace=True)

  # Calculate moving averages
  df['MA_short'] = df['Close'].rolling(window=sma).mean()
  df['MA_long'] = df['Close'].rolling(window=lma).mean()
  df['MA_xlong'] = df['Close'].rolling(window=xlma).mean()

  df = df[df.index >= start_date]
  col = tf + '_' + 'Flag'
  df[col] = 'R'
  df.loc[df['MA_short'] > df['MA_long'], col] = 'G'
  df = df[df.index > '31-Jan-2023']
  cols = [col]
  return df[df.index > '31-Jan-2023'][cols]

In [5]:

def get_stock_buy_sell(symbol):
  m_df = get_mtm_data(symbol, 'M')
  w_df = get_mtm_data(symbol, 'W')
  d_df = get_mtm_data(symbol, 'D')
  m_df['month'] = m_df.index.month
  w_df['week'] = [date.isocalendar()[1] for date in w_df.index]
  d_df['week'] = [date.isocalendar()[1] for date in d_df.index]
  d_df['month'] = d_df.index.month
  d_df = d_df.reset_index()
  cmb_df = pd.merge(d_df, w_df, on='week', how='outer')
  cmb_df = pd.merge(cmb_df, m_df, on='month', how='outer')
  cols = ['Date','D_Flag','W_Flag','M_Flag']
  cmb_df = cmb_df[cols]
  cmb_df.set_index('Date', inplace=True)
  df = yfin.Ticker(symbol+'.NS').history(period='1y', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
  df = df[df.index < back_date]
  df = df.join(cmb_df)
  df['F_Flag'] = 'R'
  df['G_Close'] = df['Close'].min()
  df.loc[(df['D_Flag'] == 'G') & (df['W_Flag'] == 'G') & (df['M_Flag'] == 'G'), 'F_Flag'] = 'G'
  df.loc[(df['F_Flag'] == 'G'), 'G_Close'] = df['Close'].max()
  df['Prev_F_Flag'] = df['F_Flag'].shift(1)
  df_reco = df[df['F_Flag'] != df['Prev_F_Flag']]
  df_reco['Signal'] = df_reco['F_Flag'].apply(lambda x: 'InSync' if x == 'G' else 'OutSync')
  df_reco.drop(['Open','Low','High','Volume'], axis=1, inplace=True)
  return df_reco.tail(1)

def get_weekly_buy_sell():
    df_buy_sell = pd.DataFrame()
    for stock_id in all_prospects:
        df_tmp = get_stock_buy_sell(stock_id)
        df_tmp['Stock'] = stock_id
        #df_buy_sell = df_buy_sell.append(df_tmp)
        df_buy_sell = pd.concat([df_buy_sell,df_tmp])
    df_buy_sell.drop(['D_Flag','W_Flag','M_Flag','G_Close','Prev_F_Flag'], axis=1, inplace=True)
    return df_buy_sell.sort_index()

df_weekly_buy_sell = get_weekly_buy_sell()
df_weekly_buy_sell.index = df_weekly_buy_sell.index.date
df_weekly_buy_sell

,Close,F_Flag,Signal,Stock
2023-09-11,921.450012,R,OutSync,RELAXO
2023-10-03,16242.900391,R,OutSync,TASTYBITE
2023-10-03,8532.099609,R,OutSync,CERA
2023-10-09,1044.150024,R,OutSync,PEL
2023-10-09,1674.800049,R,OutSync,PVRINOX
...,...,...,...,...
2024-05-10,2771.250000,R,OutSync,ASIANPAINT
2024-05-10,1183.500000,G,InSync,JCHAC
2024-05-10,39.900002,R,OutSync,SUZLON
2024-05-10,3207.399902,G,InSync,CAMS


In [6]:
cols = ['Symbol', 'InPortfolio', 'Criteria', 'Strategy']
tmp_mypf = df_mypps[cols]
tmp_mypf = tmp_mypf.rename({'Symbol': 'Stock'}, axis=1)
tmp_mypf = pd.DataFrame(tmp_mypf.groupby(['Stock', 'Criteria', 'Strategy'])['InPortfolio'].max()).reset_index()
df_weekly_buy_sell = pd.merge(df_weekly_buy_sell.reset_index(), tmp_mypf, on="Stock").set_index("index")
df_weekly_buy_sell = df_weekly_buy_sell.rename_axis('Date')
df_weekly_buy_sell

,Close,F_Flag,Signal,Stock,Criteria,Strategy,InPortfolio
Date,,,,,,,
2023-09-11,921.450012,R,OutSync,RELAXO,AV40N,NTT,DM
2023-10-03,8532.099609,R,OutSync,CERA,AV40N,ATH,NaN
2023-11-17,786.250000,R,OutSync,TTKPRESTIG,AV40N,ATH,DM
2024-01-01,1493.199951,R,OutSync,INDIGOPNTS,AV40N,NTT,DM
2024-01-01,234.550003,G,InSync,JIOFIN,AVR,BTT,NaN
...,...,...,...,...,...,...,...
2024-05-10,149.899994,G,InSync,CUB,AAR,NTT,NaN
2024-05-10,536.250000,R,OutSync,STARHEALTH,AVSP,NTT,DM
2024-05-10,1183.500000,G,InSync,JCHAC,AV40N,ATH,NaN


In [7]:
df_weekly_buy_sell[(df_weekly_buy_sell['Signal'] == 'OutSync') & (df_weekly_buy_sell['InPortfolio'].notnull())].tail(20)

,Close,F_Flag,Signal,Stock,Criteria,Strategy,InPortfolio
Date,,,,,,,
2024-02-26,101.000000,R,OutSync,JMFINANCIL,AVR,NTT,DM
2024-02-28,187.250000,R,OutSync,IBULHSGFIN,AVR,NTT,DM
2024-03-01,22.650000,R,OutSync,HATHWAY,AVR,NTT,SV
2024-03-01,428.200012,R,OutSync,VAIBHAVGBL,AVR,NTT,DM
2024-03-01,42.200001,R,OutSync,TRIDENT,AVR,NTT,DM
2024-03-18,334.149994,R,OutSync,GICRE,AVSP,NTT,DM
2024-03-26,479.850006,R,OutSync,WIPRO,AVR,NTT,DM
2024-03-26,331.250000,R,OutSync,AWL,AVSP,NTT,DM
2024-04-30,625.349976,R,OutSync,SONACOMS,AAR,NTT,DM


In [8]:
df_weekly_buy_sell[(df_weekly_buy_sell['Signal'] == 'InSync') & (df_weekly_buy_sell['InPortfolio'].isnull())].tail(20)

,Close,F_Flag,Signal,Stock,Criteria,Strategy,InPortfolio
Date,,,,,,,
2024-04-30,210.899994,G,InSync,CASTROLIND,AVR,NTT,NaN
2024-04-30,2108.699951,G,InSync,CDSL,AVR,BTT,NaN
2024-05-02,272.049988,G,InSync,SHALBY,AVSP,NTT,NaN
2024-05-02,1066.000000,G,InSync,FINCABLES,AV40N,ATH,NaN
2024-05-02,548.650024,G,InSync,DREAMFOLKS,EXP50,ATH,NaN
2024-05-02,3498.550049,G,InSync,FOSECOIND,EXP50,ATH,NaN
2024-05-02,35.900002,G,InSync,BAJAJHIND,AVR,NTT,NaN
2024-05-03,137.247543,G,InSync,DCBBANK,AAR,NTT,NaN
2024-05-10,501.899994,G,InSync,UPL,AAR,NTT,NaN


In [9]:
df_weekly_buy_sell[df_weekly_buy_sell['Stock'] == 'CUB']

,Close,F_Flag,Signal,Stock,Criteria,Strategy,InPortfolio
Date,,,,,,,
2024-05-10,149.899994,G,InSync,CUB,AAR,NTT,NaN
